In [18]:
import numpy as np
from vec3 import vec3
from scipy.linalg import null_space, cholesky, inv

## $\omega$ calculation

If we consider that the camera has square pixels and no skewness in the les, we can define $\omega$ is of the form:
$$
\omega = 
\begin{bmatrix}
\omega_1   &   0          & \omega_2 \\
0          &   \omega_1   & \omega_3 \\
\omega_2   &   \omega_3   & \omega_4 
\end{bmatrix}
$$

Where $\omega$ is defined as $\omega = (KK^{T})^{-1}$

Since the vanishing points are orthogonal by definition, we calculate the matrix $\omega$ as the null space of A for the system of equations:
$$ V_1^{T} \omega V2 = 0 $$
$$ V_1^{T} \omega V3 = 0 $$
$$ V_3^{T} \omega V2 = 0 $$

Where the vanishing points are defined as:

$$
V_1 =
\begin{bmatrix}
x_1 \\ 
y_1 \\ 
1
\end{bmatrix}
;V_2 =
\begin{bmatrix}
x_2 \\ 
y_2 \\ 
1
\end{bmatrix}
;
V_3 =
\begin{bmatrix}
x_3 \\ 
y_3 \\ 
1
\end{bmatrix}
$$

This system can be rewritten in the form of $A\omega = 0$ where $\omega = [\omega_1, \omega_2, \omega_3, \omega_4]^{T}$ is in a vectorized form and $A$ is defined as:

$$
A \omega =
\begin{bmatrix}
x_1 x_2 + y_1 y_2   &   x_1 + x_2   &   y_1 + y_2   &   1 \\
x_1 x_3 + y_1 y_3   &   x_1 + x_3   &   y_1 + y_3   &   1 \\
x_2 x_3 + y_2 y_3   &   x_2 + x_3   &   y_2 + y_3   &   1
\end{bmatrix}
\begin{bmatrix}
\omega_1 \\
\omega_2 \\
\omega_3 \\
\omega_4 \\
\end{bmatrix} =
0
$$

In [13]:
def calc_w(vp1, vp2, vp3):
    x1, y1 = vp1
    x2, y2 = vp2
    x3, y3 = vp3
    A = np.array([[(x1*x2)+(y1*y2), x2+x1, y1+y2, 1],
                   [(x1*x3)+(y1*y3), x3+x1, y1+y3, 1],
                   [(x3*x2)+(y3*y2), x2+x3, y3+y2, 1]])
    w = null_space(A)
    w /= w[3]
    return np.array([[w[0],  0,   w[1]],
                     [0,    w[0], w[2]],
                     [w[1], w[2], w[3]]], dtype=np.float32)

In [14]:
vp1 = (212, 2138)
vp2 = (-49, 42)
vp3 = (1105, 146)
W = calc_w(vp1, vp2, vp3)
W

array([[ 2.0110647e-06,  0.0000000e+00, -7.7109045e-04],
       [ 0.0000000e+00,  2.0110647e-06, -4.7431508e-04],
       [-7.7109045e-04, -4.7431508e-04,  1.0000000e+00]], dtype=float32)

## K calculation
Since $\omega$ is defined as
$$
\omega = (K K^T)^{-1}
$$

We can easily obtain the intrinsic parameters of the camer $K$ by decomposing $\omega$ through the Cholesky factorization and obtaining its inverse.

In [15]:
K = inv(cholesky(W))
K /= K[2][2]
K

array([[542.77856,  -0.     , 383.42398],
       [  0.     , 542.77856, 235.85274],
       [  0.     ,   0.     ,   1.     ]], dtype=float32)

# Stereographic Projection

We can project any point or line into a space of $\mathcal{R}^2$ delimited within a circle of radius $r$, given a sphere of radius $r$, with the center $C$ located in coordinates $[0, 0, r]$. Any point $P$ can be projected into this map to point $S$ by obtaining the vector $\mathbf{v}$ from $C$ to $P$, normalizing it and scaling by the spheres radius:
$$
S = r \frac{\mathbf{v}}{\left\lVert \mathbf{v} \right\rVert}
$$
Finally point $P$ is represented by the orthonormal projection of $S$ into the image.

In [19]:
def project_to_stereomap(p, r, w, h):
    sphere_center = vec3([w//2, h//2, r])
    v = p - sphere_center
    s = r * (v/abs(v))
    return np.array(s[:2])

array([86.60254038, 86.60254038])